In [1]:
import pandas as pd

In [2]:
train = pd.read_csv(r'final_train.csv')

In [3]:
x = train[['brand', 'model', 'model_year', 'milage', 'fuel_type','ext_col', 
        'int_col', 'accident','litre', 'automatic', 'hp', 'speed','Cylinder Info','engine type','voltage']]
y = train['price']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_ = scaler.fit_transform(x)

In [4]:
x

,brand,model,model_year,milage,fuel_type,ext_col,int_col,accident,litre,automatic,hp,speed,Cylinder Info,engine type,voltage
0,14,649,2018,74349,2,28,63,1,3.5,1,375,10,14,1,-1
1,4,49,2007,80000,2,19,12,1,3.0,0,300,6,11,1,-1
2,21,1803,2009,91491,2,194,9,1,4.2,1,300,6,5,1,-1
3,4,1779,2022,2437,1,107,28,1,3.0,2,335,-1,6,1,-1
4,40,699,2001,111000,2,263,12,1,3.8,1,200,-1,14,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,4,1776,2017,29000,2,263,28,1,4.4,1,445,8,5,1,-1
54269,3,167,2015,94634,0,19,12,0,2.0,1,220,6,10,1,-1
54270,41,424,2013,40989,2,263,12,0,3.6,2,420,-1,14,1,-1
54271,41,144,2023,1518,2,13,28,1,4.0,1,4,8,8,6,24


In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error


# 2. Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_, y, test_size=0.2, random_state=42)

# 3. Define the parameter grid for the Random Forest Regressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# 4. Perform grid search to find the best parameters
rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# 5. Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters:", best_params)

# 6. Evaluate the model using RMSE
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Best Parameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Root Mean Squared Error (RMSE): 47401.254879088316


In [4]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(x_, y, test_size=0.2, random_state=42)

# Initialize the XGBRegressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters
best_xgb_reg = xgb.XGBRegressor(**best_params)
best_xgb_reg.fit(X_train, y_train)

# Predict and evaluate
y_pred = best_xgb_reg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error: ", rmse)

Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best parameters found:  {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 200, 'subsample': 0.9}
Root Mean Squared Error:  47874.70480404242


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(x_, y, test_size=0.2, random_state=42)

# Initialize the XGBRegressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Define the parameter grid
param_grid = {
    'xgbregressor__n_estimators': [100, 200, 300],
    'xgbregressor__learning_rate': [0.01, 0.05, 0.1],
    'xgbregressor__max_depth': [3, 5, 7],
    'xgbregressor__min_child_weight': [1, 3, 5],
    'xgbregressor__subsample': [0.7, 0.8, 0.9],
    'xgbregressor__colsample_bytree': [0.7, 0.8, 0.9],
    'rfe__n_features_to_select': [5,11,15]  # Number of features to select
}

# Initialize RFE with the XGBRegressor
rfe = RFE(estimator=xgb_reg)

# Create a pipeline that combines RFE and GridSearchCV
pipeline = Pipeline(steps=[('rfe', rfe), ('xgbregressor', xgb_reg)])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Extract the best RFE and XGBRegressor from the pipeline
best_rfe = grid_search.best_estimator_.named_steps['rfe']
best_xgb_reg = grid_search.best_estimator_.named_steps['xgbregressor']

# Predict and evaluate
y_pred = best_xgb_reg.predict(best_rfe.transform(X_test))
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error: ", rmse)

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


In [16]:
from xgboost import XGBRegressor
model1 = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.05, max_depth = 3,
                     min_child_weight = 5, n_estimators = 200, subsample = 0.9)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(bootstrap = True, max_depth = 30, max_features = 'sqrt', 
                              min_samples_leaf = 4, min_samples_split = 2, n_estimators = 200)

In [5]:
test = pd.read_csv(r'final_test.csv')

In [6]:
ip = test[['brand', 'model', 'model_year', 'milage', 'fuel_type','ext_col', 
        'int_col', 'accident','litre', 'automatic', 'hp', 'speed','Cylinder Info','engine type','voltage']]
ip_ = scaler.transform(ip)

In [13]:
model.fit(x_,y)
preds = model.predict(ip_)

In [14]:
test['price']=preds

In [15]:
sub = test[['id','price']]
sub

,id,price
0,54273,23321.370469
1,54274,28381.173368
2,54275,31103.036737
3,54276,63822.930387
4,54277,36822.140626
...,...,...
36178,90451,82027.616796
36179,90452,10390.992793
36180,90453,10842.412300
36181,90454,55120.576371


In [22]:
sub.to_csv(r'cohort submission 1.csv')

In [17]:
model1.fit(x_,y)
pred_xgb = model1.predict(ip_)

In [18]:
test['xgb_price']=pred_xgb
xgb_sub = test[['id','xgb_price']]
xgb_sub

,id,xgb_price
0,54273,21894.654297
1,54274,20642.507812
2,54275,27307.714844
3,54276,59538.546875
4,54277,36779.886719
...,...,...
36178,90451,76389.406250
36179,90452,15057.608398
36180,90453,12305.201172
36181,90454,55284.613281


In [20]:
new_price = 0.5*preds + 0.5*pred_xgb
test['bagged_price']=new_price
bagged_pred = test[['id','bagged_price']]

In [21]:
bagged_pred

,id,bagged_price
0,54273,22608.012383
1,54274,24511.840590
2,54275,29205.375790
3,54276,61680.738631
4,54277,36801.013672
...,...,...
36178,90451,79208.511523
36179,90452,12724.300596
36180,90453,11573.806736
36181,90454,55202.594826


In [23]:
bagged_pred.to_csv(r'cohort submission 2.csv')

In [7]:
import xgboost as xgb
from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Initialize the XGBRegressor with best parameters found from GridSearchCV
best_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 5,
    'min_child_weight': 3,
    'subsample': 0.9,
    'colsample_bytree': 0.7
}
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', **best_params)

# Initialize the Bagging Regressor with XGBRegressor as the base estimator
bagging_regressor = BaggingRegressor(base_estimator=xgb_reg, n_estimators=10, random_state=42, n_jobs=-1)

# Train the Bagging Regressor
bagging_regressor.fit(X_train, y_train)

# Predict and evaluate
y_pred = bagging_regressor.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error with Bagging: ", rmse)

C:\Users\arjun\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Root Mean Squared Error with Bagging:  47969.51660898808


In [5]:
import xgboost as xgb
from sklearn.ensemble import BaggingRegressor

best_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 5,
    'min_child_weight': 3,
    'subsample': 0.9,
    'colsample_bytree': 0.7
}
best_xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', **best_params)

from sklearn.ensemble import RandomForestRegressor, VotingRegressor
rf_reg = RandomForestRegressor(bootstrap = True, max_depth = 30, max_features = 'sqrt', 
                              min_samples_leaf = 4, min_samples_split = 2, n_estimators = 200)

In [6]:
bagging_regressor1 = BaggingRegressor(base_estimator=best_xgb_reg, n_estimators=20, random_state=42, n_jobs=-1)
bagging_regressor2 = BaggingRegressor(base_estimator=rf_reg, n_estimators=20, random_state=42, n_jobs=-1)

In [7]:
# Initialize the Voting Regressor with XGBRegressor and RandomForestRegressor
voting_regressor = VotingRegressor([('xgb', bagging_regressor1), ('rf', bagging_regressor2)])

# Train the Voting Regressor
voting_regressor.fit(x_,y)

# Predict and evaluate the Voting Regressor
#y_pred_voting = voting_regressor.predict(ip)

C:\Users\arjun\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\arjun\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


VotingRegressor(estimators=[('xgb',
                             BaggingRegressor(base_estimator=XGBRegressor(base_score=None,
                                                                          booster=None,
                                                                          callbacks=None,
                                                                          colsample_bylevel=None,
                                                                          colsample_bynode=None,
                                                                          colsample_bytree=0.7,
                                                                          device=None,
                                                                          early_stopping_rounds=None,
                                                                          enable_categorical=False,
                                                                          eval_metric=None,
                                                                          feature_types=None,
                                                                          gamma=None,
                                                                          grow_policy=None,
                                                                          importance_type=None,
                                                                          interaction...
                                                                          missing=nan,
                                                                          monotone_constraints=None,
                                                                          multi_strategy=None,
                                                                          n_estimators=200,
                                                                          n_jobs=None,
                                                                          num_parallel_tree=None,
                                                                          random_state=None, ...),
                                              n_estimators=20, n_jobs=-1,
                                              random_state=42)),
                            ('rf',
                             BaggingRegressor(base_estimator=RandomForestRegressor(max_depth=30,
                                                                                   max_features='sqrt',
                                                                                   min_samples_leaf=4,
                                                                                   n_estimators=200),
                                              n_estimators=20, n_jobs=-1,
                                              random_state=42))])

In [9]:
import pickle

# Assuming `model` is your trained machine learning model
with open("car_price_model.pkl", "wb") as file:
    pickle.dump(voting_regressor, file)

In [ ]:
y_pred_voting

In [22]:
test['price']=y_pred_voting

In [23]:
sub2 = test[['id','price']]

In [24]:
sub2

,id,price
0,54273,21864.664080
1,54274,22688.976523
2,54275,30138.946738
3,54276,60713.738354
4,54277,39050.003548
...,...,...
36178,90451,73083.680579
36179,90452,17689.001188
36180,90453,10275.813297
36181,90454,55159.633383


In [27]:
sub2.to_csv(r'cohort submission 3.csv')